In [1]:
import numpy as np
from numba import jit

## Random tensor generator:

In [2]:
@jit(nopython=True) 
def coo_generate(shape, density=0.02):
    nnz = int(density * shape[0] * shape[1] * shape[2])
    m = np.random.choice(shape[0], nnz)
    n = np.random.choice(shape[1], nnz)
    k = np.random.choice(shape[2], nnz)
    vals = np.random.rand(nnz)
    return np.vstack((m, n, k)).T, vals, nnz

# first slow:(
coo_generate((2,2,2))

(array([], shape=(0, 3), dtype=int64), array([], dtype=float64), 0)

In [3]:
@jit(nopython=True) 
def check(coo, nnz):
    count = 0
    for i in range(nnz):
        for j in range(nnz):
            if (coo[i]==coo[j]).sum() == 3:
                count += 1
                if count > 1:
                    return "Bad"
        count = 0  

In [4]:
init_shape = (500, 600, 700)
coo, vals, nnz = coo_generate(init_shape, density=0.00002)

In [5]:
nnz

4200

## CP-ALS3:

In [6]:
@jit(nopython=True) 
def mttcrp(coo_tensor, vals, nnz, shape, mode, a, b):
    temp = np.zeros(shape=(shape[mode], a.shape[1]))
    
    if mode == 0:
        mode_a = 1 
        mode_b = 2
        
    elif mode == 1:
        mode_a = 0
        mode_b = 2
        
    else:
        mode_a = 0
        mode_b = 1
        
    for item in range(nnz):
        coord = coo_tensor[item]
        temp[coord[mode], :] += a[coord[mode_a], :] * b[coord[mode_b], :] * vals[item] 
    
    return temp

In [7]:
@jit(nopython=True) 
def cp_als(coo_tensor, vals, nnz, shape, rank=5, max_iter=200, tol=1e-8):
    a = np.random.rand(shape[0], rank)
    b = np.random.rand(shape[1], rank)
    c = np.random.rand(shape[2], rank)
    
    it = 0
    err1 = 1.0
    err2 = 0.0
    while np.abs(err1 - err2) > tol:
        v1 = b.T @ b
        v2 = c.T @ c
        v = v1 * v2
        v = np.linalg.pinv(v)
        a = mttcrp(coo_tensor, vals, nnz, shape, 0, b, c) @ v
        
        v1 = a.T @ a
        v2 = c.T @ c
        v = v1 * v2
        v = np.linalg.pinv(v)
        b = mttcrp(coo_tensor, vals, nnz, shape, 1, a, c) @ v
        
        v1 = a.T @ a
        v2 = b.T @ b
        v = v1 * v2
        v = np.linalg.pinv(v)
        c = mttcrp(coo_tensor, vals, nnz, shape, 2, a, b) @ v
        
        error = sqrt_err(coo_tensor, vals, nnz, shape, a, b, c)
        err2 = err1
        err1 = error
        #print(error)
        it += 1
        if it == max_iter:
            print("iterations over")
            break
    
    return a, b, c 

coo_generate((2, 2, 2), density=0.2)

(array([[0, 1, 0]]), array([0.53835594]), 1)

In [8]:
@jit(nopython=True) 
def sqrt_err(coo_tensor, vals, nnz, shape, a, b, c):
    result = 0.0
    for item in range(nnz):
        coord = coo_tensor[item]
        result += (vals[item] - np.sum(
            a[coord[0], :] * b[coord[1], :] * c[coord[2], :]))**2)        
    return np.sqrt(result)  

In [9]:
init_shape = (100, 100, 100)
coo, vals, nnz = coo_generate(init_shape, density=0.00002)
assert check(coo, nnz)!= "Bad"

In [10]:
nnz

20

## Experiments:

In [11]:
a, b, c = cp_als(coo, vals, nnz, init_shape, rank=200, max_iter=1_000_000)

<ipython-input-7-70b3129c5b92>:15: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  a = mttcrp(coo_tensor, vals, nnz, shape, 0, b, c) @ v
<ipython-input-7-70b3129c5b92>:21: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  b = mttcrp(coo_tensor, vals, nnz, shape, 1, a, c) @ v
<ipython-input-7-70b3129c5b92>:27: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  c = mttcrp(coo_tensor, vals, nnz, shape, 2, a, b) @ v
/home/albert/anaconda3/lib/python3.7/site-packages/numba/typing/npydecl.py:967: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  warnings.warn(NumbaPerformanceWarning(msg))
/home/albert/anaconda3/lib/python3.7/site-packages/numba/typing/npydecl.py:967: NumbaPerformanceWarning: '@' is faster on contigu

In [12]:
error = sqrt_err(coo, vals, nnz, init_shape, a, b, c)
print(error)

0.09585654830366197


In [13]:
error/np.sqrt((vals**2).sum())

0.03599308472004954

In [ ]:
#################################################################################